# Sentiment Analysis Using BERT

In this notebook, we build a binary text classifier to classify movie reviews as either positive or negative using [BERT](https://arxiv.org/abs/1810.04805), a pretrained NLP model that can be used for transfer learning on text data.  We will use the [*ktrain* library](https://github.com/amaiya/ktrain), a lightweight wrapper around Keras to help train (and deploy) neural networks.  For more information on *ktrain*, see [this Medium post](https://towardsdatascience.com/ktrain-a-lightweight-wrapper-for-keras-to-help-train-neural-networks-82851ba889c).

We will begin by installing *ktrain* and importing the required *ktrain* modules.

In [ ]:
# install ktrain
!pip3 install ktrain

     |████████████████████████████████| 25.2MB 129kB/s 
     |████████████████████████████████| 421.8MB 40kB/s 
     |████████████████████████████████| 6.7MB 54.7MB/s 
     |████████████████████████████████| 983kB 47.6MB/s 
     |████████████████████████████████| 245kB 53.8MB/s 
     |████████████████████████████████| 675kB 49.6MB/s 
     |████████████████████████████████| 471kB 56.4MB/s 
     |████████████████████████████████| 450kB 62.5MB/s 
     |████████████████████████████████| 3.9MB 48.1MB/s 
     |████████████████████████████████| 3.8MB 56.6MB/s 
     |████████████████████████████████| 890kB 46.6MB/s 
     |████████████████████████████████| 1.1MB 46.5MB/s 
  Created wheel for ktrain: filename=ktrain-0.16.3-cp36-none-any.whl size=25246180 sha256=08bc877b3ba97ccec562bf7411fe35ca9e3931076e1a8dde42203d42b5c7b9be
  Stored in directory: /root/.cache/pip/wheels/2b/24/21/a52cc4dcb0b438cf570e9d44c017761d41335d9af4c189ebad
  Created wheel for keras-bert: filename=keras_bert-0.84.0-cp36-no

In [ ]:
# import ktrain
import ktrain
from ktrain import text

In [ ]:
ktrain.__version__

'0.16.3'

Next, we will fetch and extract the IMDb movie review dataset.

In [ ]:
# download IMDb movie review dataset
import tensorflow as tf

In [ ]:
# set path to dataset
import os.path
#dataset = '/root/.keras/datasets/aclImdb'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## STEP 1:  Load and Preprocess the Dataset

The `texts_from_folder` function will load the training and validation data from the specified folder and automatically preprocess it according to BERT's requirements.  In doing so, the BERT model and vocabulary will be automatically downloaded.

In [ ]:
trn, val, preproc = text.texts_from_folder('/content/drive/My Drive/upload', 
                                          maxlen=280, 
                                          preprocess_mode='bert',
                                          train_test_names=['train', 
                                                            'test'],
                                          classes=['pos', 'neg'])

detected encoding: utf-8
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


## STEP 2:  Load a pretrained BERT model and wrap it in a `ktrain.Learner` object

This step can be condensed into a single line of code, but we execute it as two lines for clarity. (You can ignore the deprecation warnings arising from Keras 2.2.4 with TensorFlow 1.14.0.)  

In [ ]:
model = text.text_classifier('bert', trn, preproc=preproc)
learner = ktrain.get_learner(model,train_data=trn, val_data=val, batch_size=6)

Is Multi-Label? False
maxlen is 280
done.


## STEP 3:  Train and Fine-Tune the Model

We employ the `learner.fit_onecycle` method in *ktrain* that employs the use of a [1cycle learning  rate schedule](https://arxiv.org/pdf/1803.09820.pdf).  We use a learning rate of 2e-5 based on recommendations from [the original paper](https://arxiv.org/abs/1810.04805).

In [ ]:
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 1059 samples, validate on 111 samples
1059/1059 [==============================] - 162s 153ms/sample - loss: 0.1678 - accuracy: 0.9311 - val_loss: 0.4069 - val_accuracy: 0.9009
